# Notebook 12.e: A Glimpse of Calculus

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- Interpret the first difference as the average rate of change between two points.
- Discuss the "sensitivity" of a function at a given point.
- Make the intuitive leap from discrete differences to the continuous derivative.

*This notebook is under construction.